In [4]:
from os import path
import pandas as pd
import numpy as np
import json
import requests
from tqdm import tqdm
from dotenv import load_dotenv
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Định nghĩa đường dẫn
DATA_DIR = r"c:\Users\Admin\Python\ABSA_Prompting\data"
RESULT_DIR = r"c:\Users\Admin\Python\ABSA_Prompting\results"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

In [5]:
load_dotenv()

True

In [6]:
# CONFIG
ViABSA_BP_dir = path.join(DATA_DIR, 'ViABSA_Restaurant')
test_file = path.join(ViABSA_BP_dir, 'data_test.csv')
test_df = pd.read_csv(test_file)

In [7]:
aspects = [
   "AMBIENCE#GENERAL",
   "DRINKS#PRICES",
   "DRINKS#QUALITY",
   "DRINKS#STYLE&OPTIONS",
   "FOOD#PRICES",
   "FOOD#QUALITY",
   "FOOD#STYLE&OPTIONS",
   "LOCATION#GENERAL",
   "RESTAURANT#GENERAL",
   "RESTAURANT#MISCELLANEOUS",
   "RESTAURANT#PRICES",
   "SERVICE#GENERAL"
]

In [8]:
sentiment_map = {
    1: "positive",
    2: "negative",
    3: "neutral"
}

def transform_aspect_sentiment(df, start=0, end=None):
    result = [] 
    
    if end is None:
        end = len(df)

    for idx, row in df.iloc[start:end].iterrows():
        entry = {
            "id": str(idx),
            "text": row['Review'],
            "sentiments": []
        }

        for aspect in aspects:
            sentiment = row[f"{aspect}_label"]
            if sentiment == 1:  # chỉ lấy những cái có sentiment
                aspect_sentiment_value = row[aspect]
                mapped_sent = sentiment_map.get(aspect_sentiment_value, "unknown")
                if aspect_sentiment_value != 'none':
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": mapped_sent
                    })
                else:
                    # nếu cột sentiment text bị none nhưng label == 1 thì có thể log ra kiểm tra
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": "unknown"
                    })

        result.append(entry)

    return result

In [9]:
# SETUP DATA
test_df[aspects] = test_df[aspects].fillna('none')

for aspect in aspects:
    test_df[aspect + '_label'] = (test_df[aspect] != 0).astype(int)

test_json = transform_aspect_sentiment(test_df, 0, 100)
test_json[:5]

[{'id': '0',
  'text': 'Đây là 1 trong những quán mà mình thích vì vị trà đậm và thơm cũng như mùi vị đặc trưng hơn hẳn những quán khác nè  Trà sữa trân châu sợi - 46k Trà sữa pha khá ngon, vị trà chát và mùi hương khá rõ, không quá ngọt, rất đúng với gu mình  Trà đào - 45k Vị trà đào ở đây cũng đặc biệt hơn hẳn những quán khác, không phải chua ngọt như thưởng thấy mà có mùi trà rất ngon  Cà phê đá xay - 65k Món đá xay ở đây uống cũng ngon không kém trà nè, mùi vị thơm hương cà phê, vị đắng kết hợp hoàn hảo với độ béo ngọt của whipping cream, không quá đắng, cũng không quá ngọt hay lạt lẽo mà dịu nhẹ, thơm và dễ uống lắm  Trà vải thiết quan âm - 45k Trà vải có mùi vị rất thơm ngon mùi vải mà vẫn nghe rõ vị trà, có chút vị chát nhẹ mùi trà thơm rất thích, không phải chỉ toàn vị syrup vải ngọt gắt như nhiều chỗ khác. Do trà ở đây pha khá đậm nên bạn nào uống mà đang đói sẽ dễ say nha, hoặc ban đêm có thể khó ngủ à, cảnh báo trước  Trà thiết quan âm latte - 47k Ly này thì vị trà rất đậm n

In [10]:
'''
    Nếu dùng TD-IDF thì sẽ chỉ chọn dựa vào tần suất từ, không hiểu nghĩa của câu -> bị trùng lặp
    Semantic grouping để chọn example đa dạng về nội dung, không chỉ về các từ
'''
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(test_df['Review'].tolist(), show_progress_bar=True)

# KMeans clustering (70% đa dạng, 30% khó)
n_total = 5
n_diverse = int(n_total * 0.7)
n_hard = n_total - n_diverse

kmeans = KMeans(n_clusters=n_diverse, random_state=42, n_init=10)
kmeans.fit(embeddings)

# Lấy diverse example gần cluster centroid
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
diverse_examples = test_df.iloc[closest]

print(f"Selected {len(diverse_examples)} diverse examples:")
for i, row in diverse_examples.iterrows():
    print(f"\nExample {i}:\n{row['Review']}")

Batches: 100%|██████████| 16/16 [00:13<00:00,  1.20it/s]


Selected 3 diverse examples:

Example 345:
Món ăn ở đây thì ngon, nhưng giá cả hơi cao. Không gian thì rất ok, phục vụ cũng nhiệt tình, chỉ có mấy anh giữ xe là hơi chán. Nhưng nếu ai muốn thử một lần sang chảnh thì có thể đến đây để tận hưởng.   Bánh bí đỏ: 48k  Bánh cuốn chiên XO: 40k  Gỏi sò huyết: 98k  Lưỡi vị nướng: 139k  Vịt quay Bắc Kinh: 600k

Example 74:
Hôm bữa mình đi có đi ăn bánh Flan trên con đường này luôn, chạy qua thấy chỗ bán bánh đúc nên tấp vào ăn thử. Mặt tiền hơi nhỏ, để chừng 4 chiếc xe là hông có đường vô luôn, bên trong không gian sạch sẽ có bàn ngồi thoải mái, có bình trà đá miễn phí nữa :)   Một chén chỉ có 15k thôi mà thịt thà cũng khá nhiều ngập mặt chứ không ít đâu nha. Bột bánh ăn cũng bình thường thôi, không được dẻo cho lắm, ăn hết chén này là ngán luôn đó. Nhân có thịt bằm với nấm mèo thêm hành phi thơm thơm nữa, nước mắm cũng vừa ăn có phần hơi ngọt tí mà mình thấy vậy là ngon rồi.  Hành phi giòn rụm luôn nè mấy chế ơi :D Ớt cho bạn nào thích ăn cay N

In [11]:
def select_few_shot_examples(df, text_column, n_total=5, model_name='all-MiniLM-L6-v2', random_state=42):
    n_diverse = int(n_total * 0.7)
    n_hard = n_total - n_diverse

    model = SentenceTransformer(model_name)
    embeddings = model.encode(df[text_column].tolist(), show_progress_bar=True)

    kmeans = KMeans(n_clusters=n_diverse, random_state=random_state, n_init=10)
    kmeans.fit(embeddings)

    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
    diverse_df = df.iloc[closest]

    all_indices = np.arange(len(df))
    remaining_indices = list(set(all_indices) - set(closest))
    hard_df = df.iloc[remaining_indices].sample(n=n_hard, random_state=random_state)

    return diverse_df.reset_index(drop=True), hard_df.reset_index(drop=True)

# SET-UP FEW-SHOT EXAMPLES
diverse, hard = select_few_shot_examples(test_df, text_column='Review', n_total=5)

few_shot_json =  transform_aspect_sentiment(diverse, 0, 100) + transform_aspect_sentiment(hard, 0, 100)


Batches: 100%|██████████| 16/16 [00:13<00:00,  1.20it/s]


In [12]:
def evaluate_aspect_sentiment(ground_truth, predictions):
    # Chuẩn hóa dữ liệu thành list các tuple để so sánh
    true_aspects = []
    pred_aspects = []

    true_aspect_sentiments = []
    pred_aspect_sentiments = []

    for gt_entry, pred_entry in zip(ground_truth, predictions):
        # ground truth: list of sentiments
        gt_sents = gt_entry['sentiments']
        gt_aspect_set = set()
        gt_aspect_sentiment_set = set()

        for item in gt_sents:
            gt_aspect_set.add(item['aspect'])
            gt_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        true_aspects.append(gt_aspect_set)
        true_aspect_sentiments.append(gt_aspect_sentiment_set)

        # prediction: list of results
        pred_sents = pred_entry['results']
        pred_aspect_set = set()
        pred_aspect_sentiment_set = set()

        for item in pred_sents:
            pred_aspect_set.add(item['aspect'])
            pred_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        pred_aspects.append(pred_aspect_set)
        pred_aspect_sentiments.append(pred_aspect_sentiment_set)

    # Tính theo micro-F1 (gộp hết lại)
    all_true_aspects = set.union(*true_aspects) if true_aspects else set()
    all_pred_aspects = set.union(*pred_aspects) if pred_aspects else set()

    tp_aspect = sum(len(gt & pred) for gt, pred in zip(true_aspects, pred_aspects))
    fp_aspect = sum(len(pred - gt) for gt, pred in zip(true_aspects, pred_aspects))
    fn_aspect = sum(len(gt - pred) for gt, pred in zip(true_aspects, pred_aspects))

    precision_aspect = tp_aspect / (tp_aspect + fp_aspect + 1e-8)
    recall_aspect = tp_aspect / (tp_aspect + fn_aspect + 1e-8)
    f1_aspect = 2 * precision_aspect * recall_aspect / (precision_aspect + recall_aspect + 1e-8)

    # Tính cho sentiment classification
    tp_sentiment = sum(len(gt & pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fp_sentiment = sum(len(pred - gt) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fn_sentiment = sum(len(gt - pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))

    precision_sentiment = tp_sentiment / (tp_sentiment + fp_sentiment + 1e-8)
    recall_sentiment = tp_sentiment / (tp_sentiment + fn_sentiment + 1e-8)
    f1_sentiment = 2 * precision_sentiment * recall_sentiment / (precision_sentiment + recall_sentiment + 1e-8)

    return {
        "Aspect Detection F1": f1_aspect,
        "Sentiment Classification F1": f1_sentiment
    }


In [13]:
# Setup Grok API
def call_grok_api(prompt):
    url = "https://api.x.ai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.getenv('GROK_API_KEY')}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "grok-3",
        "messages": [
            {"role": "system", "content": "You are an AI assistant that extracts aspects and their sentiments from text."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0
    }
    
    try:
        response = requests.post(url, headers=headers, json=data, timeout=30)
        if response.status_code == 200:
            result = response.json()
            return result['choices'][0]['message']['content']
        else:
            print(f"API Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"Request Error: {e}")
        return None

def create_clustering_prompt(text, few_shot_examples):
    prompt = "You are an AI assistant that extracts aspects and their sentiments from text.\n\n"
    
    # Thêm few-shot examples
    for ex in few_shot_examples:
        prompt += f"Extract aspects and sentiments from the following review:\n{ex['text']}\n"
        prompt += f"Result: {json.dumps({'results': ex['sentiments']}, ensure_ascii=False)}\n\n"
    
    # Thêm câu hỏi hiện tại
    prompt += f"Extract aspects and sentiments from the following review:\n{text}\n"
    prompt += f"Available aspects: {', '.join(aspects)}\n"
    prompt += "Available sentiments: positive, negative, neutral\n"
    prompt += "Return ONLY a valid JSON object in this exact format:\n"
    prompt += '{"results": [{"aspect": "aspect_name", "sentiment": "sentiment_value"}]}'
    
    return prompt

def extract_with_grok_clustering(text, few_shot_examples):
    prompt = create_clustering_prompt(text, few_shot_examples)
    response_text = call_grok_api(prompt)
    
    if response_text:
        try:
            # Tìm JSON trong response
            start_idx = response_text.find('{')
            end_idx = response_text.rfind('}') + 1
            
            if start_idx != -1 and end_idx != -1:
                json_str = response_text[start_idx:end_idx]
                parsed_output = json.loads(json_str)
                return parsed_output
            else:
                return {"results": []}
        except json.JSONDecodeError:
            print(f"JSON Parse Error: {response_text}")
            return {"results": []}
    else:
        return {"results": []}

predictions = []

# Dự đoán
for data in tqdm(test_json):
    prediction = extract_with_grok_clustering(data['text'], few_shot_json)
    predictions.append(prediction)

100%|██████████| 100/100 [03:57<00:00,  2.38s/it]


In [14]:
scores = evaluate_aspect_sentiment(test_json, predictions)
print(scores)

{'Aspect Detection F1': 0.8004484254923183, 'Sentiment Classification F1': 0.6950672595753987}


In [15]:
result_file = path.join(RESULT_DIR, 'ViABSA_BP_Few-Restaurant-Grok.json')
with open(result_file, 'w') as f:
    json.dump(predictions, f, indent=4, ensure_ascii=False)